# Appendix 10.1: Chaining Prompts

- [Lesson](#lesson)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# Has been rewritten to take in a messages list of arbitrary length
def get_completion(messages, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages
    )
    return message.content[0].text

---

## Lesson

The saying goes, "Writing is rewriting." It turns out, **Claude can often improve the accuracy of its response when asked to do so**!

There are many ways to prompt Claude to "think again". The ways that feel natural to ask a human to double check their work will also generally work for Claude. (Check out our [prompt chaining documentation](https://docs.anthropic.com/claude/docs/chain-prompts) for further examples of when and how to use prompt chaining.)

### Examples

In this example, we ask Claude to come up with ten words... but one or more of them isn't a real word.

In [2]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Sure! Here are ten words that all end with the exact letters "ab":

1. Grab
2. Crab
3. Snub
4. Stab
5. Tab
6. Fab
7. Jab
8. Lap
9. Map
10. Rob

These words vary in meaning and usage, but they all share the ending "ab".


**Asking Claude to make its answer more accurate** fixes the error! 

Below, we've pulled down Claude's incorrect response from above and added another turn to the conversation asking Claude to fix its previous answer.

In [3]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': 'Sure! Here are ten words that all end with the exact letters "ab":\n\n1. Grab\n2. Crab\n3. Snub\n4. Stab\n5. Tab\n6. Fab\n7. Jab\n8. Lap\n9. Map\n10. Rob\n\nThese words vary in meaning and usage, but they all share the ending "ab".'}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words."}]

------------------------------------- Claude's response -------------------------------------
Certainly! Here are ten made-up or non-real words that end with "ab":

1. Glub
2. Flub
3. Clubb
4. Snubb
5. Jubb
6. Tubb
7. Zubb
8. Pubb
9. Rabb
10. Mubb

These are fictional words created for the purpose of your request.


But is Claude revising its answer just because we told it to? What if we start off with a correct answer already? Will Claude lose its confidence? Here, we've placed a correct response in the place of `first_response` and asked it to double check again.

In [4]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words."}]

------------------------------------- Claude's response -------------------------------------
Sure! Here are 10 "words" ending in "ab" where the first part is not a real word:

1. Zib
2. Glib
3. Jib
4. Kib
5. Lib
6. Mib
7. Nib
8. Pib
9. Rib
10. Tib

These are made-up words, but they follow the pattern of ending in "ab".


You may notice that if you generate a respnse from the above block a few times, Claude leaves the words as is most of the time, but still occasionally changes the words even though they're all already correct. What can we do to mitigate this? Per Chapter 8, we can give Claude an out! Let's try this one more time.

In [5]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."}]

------------------------------------- Claude's response -------------------------------------
Let's check each word to ensure they are valid English words:

1. Cab - Real word
2. Dab - Real word
3. Grab - Real word
4. Gab - Real word
5. Jab - Real word
6. Lab - Real word
7. Nab - Real word
8. Slab - Real word
9. Tab - Real word
10. Blab - Real word

Since all the words are real English words, here is the original list:

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab


Try generating responses from the above code a few times to see that Claude is much better at sticking to its guns now.

You can also use prompt chaining to **ask Claude to make its responses better**. Below, we asked Claude to first write a story, and then improve the story it wrote. Your personal tastes may vary, but many might agree that Claude's second version is better.

First, let's generate Claude's first version of the story.

In [6]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In the early morning light, Lily would slip on her sneakers and disappear down the quiet street, her heart pounding with each stride. Each run was a dance with the sunrise, her breaths deep and rhythmic as she embraced the freedom of movement. By the time she reached the park, her hair tousled and face flushed, she felt invigorated, ready to take on any challenge the day might bring.


Now let's have Claude improve on its first draft.

In [7]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Write a three-sentence short story about a girl who likes to run.'}, {'role': 'assistant', 'content': 'In the early morning light, Lily would slip on her sneakers and disappear down the quiet street, her heart pounding with each stride. Each run was a dance with the sunrise, her breaths deep and rhythmic as she embraced the freedom of movement. By the time she reached the park, her hair tousled and face flushed, she felt invigorated, ready to take on any challenge the day might bring.'}, {'role': 'user', 'content': 'Make the story better.'}]

------------------------------------- Claude's response -------------------------------------
In the early morning light, Lily would slip on her sneakers and disappear down the quiet street, her heart pounding with each stride as the cool breeze danced through her hair. Each run was a dance with the sunrise, her breaths de

This form of substitution is very powerful. We've been using substitution placeholders to pass in lists, words, Claude's former responses, and so on. You can also **use substitution to do what we call "function calling," which is asking Claude to perform some function, and then taking the results of that function and asking Claude to do even more afterward with the results**. It works like any other substitution. More on this in the next appendix.

Below is one more example of taking the results of one call to Claude and plugging it into another, longer call. Let's start with the first prompt (which includes prefilling Claude's response this time).

In [8]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Find all names from the below text:\n\n"Hey, Jesse. It\'s me, Erin. I\'m calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too.'}, {'role': 'assistant', 'content': '<names>'}]

------------------------------------- Claude's response -------------------------------------
Joey, Jesse, Keisha, Mel, Erin</names>


Let's pass this list of names into another prompt.

In [9]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Find all names from the below text:\n\n"Hey, Jesse. It\'s me, Erin. I\'m calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too.'}, {'role': 'assistant', 'content': '<names>\nJoey, Jesse, Keisha, Mel, Erin</names>'}, {'role': 'user', 'content': 'Alphabetize the list.'}]

------------------------------------- Claude's response -------------------------------------
Here is the alphabetized list of names:

- Joey
- Jesse
- Keisha
- Mel
- Erin


Now that you've learned about prompt chaining, head over to Appendix 10.2 to learn how to implement function calling using prompt chaining.

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))